# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [99]:
import pandas as pd

# Загрузка данных
recipes = pd.read_csv('recipes_sample.csv', index_col='id')
reviews = pd.read_csv('reviews_sample.csv', index_col=0)

# Оставление только нужных столбцов в таблице с рецептами
recipes = recipes[['name', 'minutes', 'submitted', 'description', 'n_ingredients']]

# Вывод первых 5 строк таблицы
print(recipes.head())

                                           name  minutes   submitted  \
id                                                                     
44123     george s at the cove  black bean soup       90  2002-10-25   
67664        healthy for them  yogurt popsicles       10  2003-07-26   
38798              i can t believe it s spinach       30  2002-08-29   
35173                      italian  gut busters       45  2002-07-27   
84797  love is in the air  beef fondue   sauces       25  2004-02-23   

                                             description  n_ingredients  
id                                                                       
44123  an original recipe created by chef scott meska...           18.0  
67664  my children and their friends ask for my homem...            NaN  
38798            these were so go, it surprised even me.            8.0  
35173  my sister-in-law made these for us at a family...            NaN  
84797  i think a fondue is a very romantic casual d

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [100]:
from openpyxl import Workbook

# Загрузка данных
recipes = pd.read_csv('recipes_sample.csv', index_col='id')
reviews = pd.read_csv('reviews_sample.csv', index_col=0)

# Случайный выбор 5% строк из каждой таблицы
recipes_sample = recipes.sample(frac=0.05)
reviews_sample = reviews.sample(frac=0.05)

# Создание файла Excel
writer = pd.ExcelWriter('recipes.xlsx', engine='openpyxl')

# Сохранение таблиц на разные листы
recipes_sample.to_excel(writer, sheet_name='Рецепты')
reviews_sample.to_excel(writer, sheet_name='Отзывы')

# Назначение названий листов
workbook = writer.book
worksheet1 = writer.sheets['Рецепты']
worksheet2 = writer.sheets['Отзывы']

# Сохранение файла
writer.save()

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [101]:
import xlwings as xw

# Загрузка данных из файла
wb = xw.Book('recipes.xlsx')
recipes_sheet = wb.sheets['Рецепты']
recipes_df = recipes_sheet.used_range.options(pd.DataFrame, header=1).value


# Рассчет времени выполнения рецепта в секундах
recipes_df['seconds_assign'] = recipes_df['minutes'] * 60

# Добавление столбца seconds_assign на лист Рецепты при помощи присваивания массива значений
recipes_sheet.range((1, recipes_df.shape[1] + 1)).value = [['seconds_assign']] + recipes_df['seconds_assign'].values.reshape(-1, 1).tolist()

# Сохранение изменений в файле
wb.save()

# Вывод на экран результатов изменений
print(recipes_df[['minutes','seconds_assign']])

          minutes  seconds_assign
id                               
97996.0      30.0          1800.0
490784.0    250.0         15000.0
287868.0    110.0          6600.0
411505.0    120.0          7200.0
111720.0    140.0          8400.0
...           ...             ...
227691.0     50.0          3000.0
48494.0      45.0          2700.0
154364.0     30.0          1800.0
190812.0    120.0          7200.0
495291.0     70.0          4200.0

[1500 rows x 2 columns]


4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [102]:
wb = xw.Book('recipes.xlsx')
recipes_sheet = wb.sheets['Рецепты']

# Добавление формулы в ячейку F1
recipes_sheet.range('J1').value = 'seconds_formula'
recipes_sheet.range('J2:J1501').formula = '=C2*60'  # применение формулы для всего столбца

# Сохранение изменений в файле
wb.save()

# Вывод на экран результатов изменений
print(recipes_sheet.range('I1:J6').options(pd.DataFrame, header=1).value)

                seconds_formula
seconds_assign                 
1800.0                   1800.0
15000.0                 15000.0
6600.0                   6600.0
7200.0                   7200.0
8400.0                   8400.0


6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [103]:
import xlwings as xw

# Определение цветов для раскраски
green = (0, 255, 0)
yellow = (255, 255, 0)
red = (255, 0, 0)

# Получение столбца минут
minutes_range = recipes_sheet.range('C2:C1501')

# Применение условного форматирования
for cell in minutes_range:
    minutes = cell.value
    if minutes < 5:
        color = green
    elif minutes < 10:
        color = yellow
    else:
        color = red
    cell.color = color

# Сохранение изменений в файле
wb.save()

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [105]:
# В данной задаче я ищу кол-во отзывов не в общем, а из рандомно взятых отзывов, помещенных на лист Отзывы

# Добавление заголовка для нового столбца
recipes_sheet.range('K1').value = 'n_reviews'

# Добавление формулы в столбец F для каждой строки с рецептом
for i in range(2, 1502):
    recipe_id = recipes_sheet.range('A' + str(i)).value
    formula = '=COUNTIF(Отзывы!C:C, A{})'.format(i)
    recipes_sheet.range('K' + str(i)).formula = formula

# Сохранение изменений в файле
wb.save()

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [108]:
def validate():
    # Открываем файл и выбираем листы
    wb = xw.Book('recipes.xlsx')
    reviews_sheet = wb.sheets['Отзывы']
    recipes_sheet = wb.sheets['Рецепты']

    # Проверяем каждую строку
    for i in range(2, reviews_sheet.range('A1').current_region.last_cell.row):
        rating = reviews_sheet.range(f'E{i}').value
        recipe_id = reviews_sheet.range(f'C{i}').value

        # Проверяем рейтинг
        if rating is None or not isinstance(rating, (int, float)) or rating < 0 or rating > 5:
            reviews_sheet.range(f'A{i}:F{i}').color = (255, 0, 0)  # Выделяем строку красным цветом
            continue

        # Проверяем, есть ли рецепт с таким id
        if recipe_id not in recipes_sheet.range('A2:A1501').value:
            reviews_sheet.range(f'A{i}:F{i}').color = (255, 0, 0)  # Выделяем строку красным цветом
            continue

    # Сохраняем изменения
    wb.save()

validate()

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

In [130]:
import csv
import xlwings as xw

# Считываем данные из csv-файла
with open('recipes_model.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter="\t")
    data = list(reader)

# Создаем новую книгу в Excel
wb = xw.Book()
ws = wb.sheets.add('Модель')

# Записываем данные на лист Модель
ws.range('A2:I18').value = data

# Сохраняем книгу
wb.save('recipes_model.xlsx')

![](LR4_9.png)

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [151]:
import xlwings as xw

# Загрузка данных из файла
wb = xw.Book('recipes_model.xlsx')
model_sheet = wb.sheets['Модель']

# Заливка цвета для заголовков 
model_sheet.range('A2:I2').color = ('00ccff')

# Применение автоподбора ширины столбца
model_sheet.autofit(axis="columns")

# Шрифт заголовков полужирный
model_sheet.range('A2:I2').font.bold = True

# Добавление таблице автофильтра
model_sheet.show_autofilter = True

# Сохранение изменений в файле
wb.save()

![](LR4_11.png)

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 